<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Quantum_Mechanics_Text_Adventure_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import IPython.display as display
from google.colab import userdata
from google.colab import output
import requests
import json
import time # For exponential backoff
import asyncio # Import asyncio
import nest_asyncio # Import nest_asyncio

# Apply nest_asyncio to allow asyncio to run inside a running event loop
nest_asyncio.apply()

# Google Colab Specific API Key Setup
# This part assumes you are running in Google Colab and have stored your API key
try:
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    print("Not in Google Colab, or 'userdata' module not found. Please set GOOGLE_API_KEY manually.")
    GOOGLE_API_KEY = None # Fallback for local execution

LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20" # Explicitly use the preferred model

# Global Game State
game_chat_history = []
current_story_description = "Welcome, young quantum explorer! Click 'Start New Game' to begin your adventure into the tiny world of atoms! ⚛️"
current_choices = []
game_over_status = False
loading_message = "Exploring quantum realms... Please wait. "
error_message = ""
info_message = ""

# LLM API Call Function
async def call_llm(prompt_text, max_retries=5, initial_delay=1.0):
    """
    Calls the Gemini LLM API to get simulation parameters.
    Implements exponential backoff for retries.
    """
    global game_chat_history, loading_message, error_message, info_message

    # Clear previous messages
    error_message = ""
    info_message = ""

    # Check if API key is available
    if not GOOGLE_API_KEY:
        error_message = "Google API Key not available. Cannot call LLM."
        print(f"ERROR: {error_message}")
        update_game_ui(is_loading=False) # Ensure UI updates with error
        return None

    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{LLM_MODEL_NAME}:generateContent?key={GOOGLE_API_KEY}"
    headers = {"Content-Type": "application/json"}

    # Add the prompt to history regardless of whether it's an initial prompt or an action.
    # The prompt_text is always the user's "turn" in this context.
    game_chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})

    payload = {
        "contents": game_chat_history,
        "generationConfig": {
            "responseMimeType": "application/json",
            "responseSchema": {
                "type": "OBJECT",
                "properties": {
                    "description": {"type": "STRING", "description": "The current story description of the quantum adventure."},
                    "choices": {
                        "type": "ARRAY",
                        "items": {"type": "STRING"},
                        "description": "List of possible actions/choices for the player in the quantum world."
                    },
                    "gameover": {"type": "BOOLEAN", "description": "True if the game is over (e.g., player learned a concept, or failed a quantum challenge)."}
                },
                "required": ["description", "choices", "gameover"]
            }
        }
    }

    retries = 0
    delay = initial_delay

    while retries < max_retries:
        try:
            # Indicate loading in the UI *before* the actual request
            update_game_ui(is_loading=True)
            response = requests.post(api_url, headers=headers, data=json.dumps(payload))
            response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
            result = response.json()

            if result and result.get("candidates") and result["candidates"][0]["content"].get("parts"):
                llm_data_str = result["candidates"][0]["content"]["parts"][0].get("text")
                if not llm_data_str:
                    error_message = "LLM response missing expected 'text' content."
                    print(f"ERROR: {error_message}")
                    return None

                # Ensure it's a string that can be parsed as JSON
                if isinstance(llm_data_str, dict): # Handle cases where LLM might return dict directly
                    parsed_json = llm_data_str
                else:
                    parsed_json = json.loads(llm_data_str)

                # Add LLM's response to chat history
                game_chat_history.append({"role": "model", "parts": [{"text": llm_data_str}]})
                return parsed_json
            else:
                error_message = f"LLM response missing expected content structure. Result: {result}"
                print(f"ERROR: {error_message}")
                if "blocked_reason" in str(result): # Example of checking for safety blocking
                    error_message = "LLM response was blocked, potential safety issue."
                    print(f"ERROR: {error_message}")
                return None

        except requests.exceptions.RequestException as e:
            error_message = f"API request failed (Attempt {retries + 1}/{max_retries}): {e}"
            print(f"ERROR: {error_message}")
            if response is not None:
                print(f"Response status: {response.status_code}, Body: {response.text[:500]}...")
            retries += 1
            if retries < max_retries:
                await asyncio.sleep(delay)
                delay *= 2 # Exponential backoff
        except json.JSONDecodeError as e:
            error_message = f"Error parsing JSON response from LLM: {e}. Raw response: {response.text[:500]}..."
            print(f"ERROR: {error_message}")
            return None
        except Exception as e:
            error_message = f"An unexpected error occurred during API call: {e}"
            print(f"ERROR: {error_message}")
            return None

    error_message = f"Failed to get LLM parameters after {max_retries} retries."
    print(f"ERROR: {error_message}")
    return None

# Game Logic Functions (Python Callbacks)

def initial_game_prompt():
    """
    Generates the initial prompt for the LLM to set up the quantum mechanics game scenario.
    """
    return """You are the game master for a text adventure designed to teach basic quantum mechanics concepts to children aged 8-12.
    Generate an initial scenario where a child protagonist discovers a magical device that lets them shrink down to the quantum level.
    The language should be simple, engaging, and suitable for children.
    Introduce a very basic quantum concept (e.g., things being "blurry" or "wavy" at a tiny scale, or electrons jumping).
    Provide a 'description' (a few sentences, 100-200 words) and a 'choices' array (2-4 child-friendly options).
    Set 'gameOver' to false.

    Example: {"description": "You found a shimmering pocket watch that glows with strange colors. When you click it, everything around you starts to wiggle and blur! You're shrinking! Your cat looks like a giant fluffy mountain, and then you're even smaller, down to the size of a dust speck! The air around you feels... bouncy, and tiny glowing dots zip past like super-fast fireflies. What do you do?", "choices": ["Touch a glowing dot.", "Look for a way to get bigger.", "Try to understand what's happening."], "gameover": false}
    """

async def start_new_game():
    """
    Resets the game state and initiates a new quantum adventure scenario.
    """
    global game_chat_history, current_story_description, current_choices, game_over_status, error_message, info_message
    game_chat_history = [] # Reset chat history
    game_over_status = False
    error_message = ""
    info_message = ""
    current_story_description = loading_message
    current_choices = []
    update_game_ui(is_loading=True) # Show loading while fetching

    scenario = await call_llm(initial_game_prompt())

    if scenario:
        current_story_description = scenario['description']
        current_choices = scenario['choices']
        game_over_status = scenario['gameover']
        if game_over_status:
            info_message = "Game Over! Click 'Start New Game' to play again."
    else:
        current_story_description = "Failed to start new game. Please check for API key errors or try again."
        current_choices = []
        game_over_status = True # End game on error
        error_message = "Error: Could not retrieve initial scenario."
    update_game_ui(is_loading=False)

async def handle_player_action(player_action):
    """
    Processes the player's action and updates the game story based on LLM response.
    """
    global current_story_description, current_choices, game_over_status, error_message, info_message

    if not player_action.strip():
        error_message = "Please enter your action or click a choice button."
        update_game_ui(is_loading=False) # Ensure loading is off and error is displayed
        return

    # Add player's action to history for display
    current_story_description += f'\n\nYou chose: "{player_action}"' # Append action to story description for immediate feedback
    update_game_ui(is_loading=True) # Show loading while fetching

    # The prompt for advancing the story, focusing on quantum mechanics concepts.
    game_prompt = f"""
    You are the game master for a text adventure designed to teach basic quantum mechanics concepts to children aged 8-12.
    The player chose: "{player_action}".
    Given the previous story and the player's choice, advance the narrative, keeping the language simple and engaging.
    Introduce or reinforce a specific quantum concept relevant to the choice (e.g., superposition, quantum leap, wave-particle duality, uncertainty principle, tunneling, entanglement, quantum foam, zero-point energy). Explain it simply through the story.
    Provide a 'description' (a few sentences, 100-200 words) that advances the story, incorporating the chosen quantum concept.
    Provide a 'choices' array (2-4 distinct child-friendly options for the player, each a short phrase).
    Set 'gameOver' to true if the story reaches a clear end where a quantum concept is understood or a challenge is completed (success or failure).
    If the game ends, the description should clearly state the outcome (e.g., "You now understand how tiny particles can be in two places at once! Game Over!").

    Remember to use simple analogies and avoid complex jargon. Focus on making the quantum concept feel like a magical or surprising part of their adventure.
    """

    scenario = await call_llm(game_prompt)

    if scenario:
        current_story_description = scenario['description']
        current_choices = scenario['choices']
        game_over_status = scenario['gameover']
        if game_over_status:
            info_message = "Game Over! Click 'Start New Game' to play again."
    else:
        current_story_description = "An error occurred while continuing the adventure. Please try a different action or restart."
        current_choices = []
        game_over_status = True # End game on error
        error_message = "Error: Could not advance scenario."
    update_game_ui(is_loading=False)

# --- UI Generation Function ---
def update_game_ui(is_loading=False):
    """
    Updates the game's HTML user interface in the Google Colab output.
    """
    global current_story_description, current_choices, game_over_status, loading_message, error_message, info_message

    choices_html = ""
    if not game_over_status and not is_loading:
        for i, choice in enumerate(current_choices):
            # Pre-process the choice to escape single quotes for JavaScript
            # A single quote ' needs to become \' in JavaScript.
            escaped_choice_for_js = choice.replace("'", "\\'")
            choices_html += f"""
            <button
                class="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded-full m-2 transition duration-300 ease-in-out transform hover:scale-105 shadow-lg"
                onclick="google.colab.kernel.invokeFunction('handle_player_action', ['{escaped_choice_for_js}'], {{}})"
            >
                {choice}
            </button>
            """
    # Message area for errors or info
    message_area_class = "hidden"
    message_content = ""
    if error_message:
        message_area_class = "bg-red-700 text-white p-3 rounded-lg text-center"
        message_content = error_message
    elif info_message:
        message_area_class = "bg-blue-700 text-white p-3 rounded-lg text-center"
        message_content = info_message

    # Conditionally generate the input area HTML
    input_area_html = ""
    if not game_over_status and not is_loading:
        input_area_html = f"""
        <div class="input-area">
            <input
                type="text"
                id="player-input"
                placeholder="Or type your own action here..."
                onkeydown="if(event.key === 'Enter') google.colab.kernel.invokeFunction('handle_player_action', [document.getElementById('player-input').value], {{}}); return event.key !== 'Enter';"
                class="rounded-lg border border-gray-600 focus:ring-blue-500 focus:border-blue-500"
            >
            <button
                class="bg-green-500 hover:bg-green-700 text-white font-bold py-2 px-6 rounded-full transition duration-300 ease-in-out transform hover:scale-105 shadow-lg"
                onclick="google.colab.kernel.invokeFunction('handle_player_action', [document.getElementById('player-input').value], {{}})"
            >
                Go!
            </button>
        </div>
        """

    html_content = f"""
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;700&display=swap" rel="stylesheet">
    <style>
        body {{
            font-family: 'Inter', sans-serif;
            background-color: #1a202c; /* Dark background */
            color: #e2e8f0; /* Light text */
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            padding: 20px;
            box-sizing: border-box;
        }}
        .game-container {{
            background-color: #2d3748; /* Darker container */
            border-radius: 20px;
            padding: 30px;
            max-width: 800px;
            width: 100%;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.5);
            text-align: center;
            display: flex;
            flex-direction: column;
            gap: 20px;
        }}
        .title {{
            font-size: 2.5rem;
            font-weight: 700;
            color: #63b3ed; /* Blue title */
            margin-bottom: 10px;
            display: flex;
            align-items: center;
            justify-content: center;
        }}
        .subtitle {{
            font-size: 1.5rem;
            color: #90cdf4; /* Lighter blue subtitle */
            margin-bottom: 20px;
        }}
        .story-box {{
            background-color: #4a5568; /* Even darker for story */
            border-radius: 15px;
            padding: 20px;
            min-height: 150px;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 1.1rem;
            line-height: 1.6;
            text-align: left;
            overflow-y: auto;
            max-height: 300px; /* Limit height for scrollability */
        }}
        .choices-container {{
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            gap: 10px;
            margin-top: 20px;
        }}
        .input-area {{
            display: flex;
            flex-direction: column;
            gap: 10px;
            margin-top: 20px;
            align-items: center;
        }}
        .input-area input {{
            width: calc(100% - 20px);
            max-width: 400px;
            padding: 12px 15px;
            border-radius: 10px;
            border: 1px solid #718096;
            background-color: #2d3748;
            color: #e2e8f0;
            font-size: 1rem;
        }}
        .input-area input:focus {{
            outline: none;
            border-color: #63b3ed;
            box-shadow: 0 0 0 3px rgba(99, 179, 237, 0.5);
        }}
        .control-buttons {{
            display: flex;
            justify-content: center;
            gap: 15px;
            margin-top: 20px;
        }}
        .message-box {{
            margin-top: 20px;
            padding: 15px;
            border-radius: 10px;
            font-weight: bold;
        }}
        .hidden {{
            display: none;
        }}
        /* Responsive adjustments */
        @media (max-width: 600px) {{
            .game-container {{
                padding: 20px;
            }}
            .title {{
                font-size: 2rem;
            }}
            .subtitle {{
                font-size: 1.2rem;
            }}
            .story-box {{
                font-size: 1rem;
            }}
        }}
    </style>

    <div class="game-container">
        <div class="title">
            🔬 Quantum Quest! ✨
        </div>
        <div class="subtitle">
            An Adventure into the Tiny World
        </div>

        <div class="story-box">
            {current_story_description}
            {is_loading and f'<div class="text-gray-400 mt-4">{loading_message}</div>' or ''}
        </div>

        <div class="choices-container">
            {choices_html if not is_loading else ''}
        </div>

        {input_area_html}

        <div class="control-buttons">
            <button
                class="bg-purple-600 hover:bg-purple-800 text-white font-bold py-2 px-6 rounded-full transition duration-300 ease-in-out transform hover:scale-105 shadow-lg"
                onclick="google.colab.kernel.invokeFunction('start_new_game', [], {{}})"
            >
                Start New Game
            </button>
        </div>

        <div class="{message_area_class}">
            {message_content}
        </div>
    </div>
    """
    # Clear the cell and display new HTML content
    display.clear_output(wait=True)
    display.display(display.HTML(html_content))

# Main Execution

# Define synchronous wrappers to call the async functions
def sync_start_new_game():
    asyncio.run(start_new_game())

def sync_handle_player_action(player_action):
    asyncio.run(handle_player_action(player_action))

# Register Python functions as callbacks callable from JavaScript
# These must be called after the functions are defined.
output.register_callback('start_new_game', sync_start_new_game)
output.register_callback('handle_player_action', sync_handle_player_action)

# Initial display of the game UI
update_game_ui()